In [1]:
# Import packages
import pandas as pd
import numpy as np
import datetime
from collections import defaultdict
import nltk
import random

In [3]:
stemmer = nltk.stem.snowball.SnowballStemmer("english")
data_dir = "/Users/yangsong/Desktop/Projects/gitrepo_songyang0716/Topic_Modeling/reviews_small.txt"
np.random.seed(666)

# read review texts
reviews = []
f = open(data_dir, "r")
for review in f:
    reviews.append(review)
random.shuffle(reviews)

In [4]:
# process text
# tokenize, lower, remove stop words, stem, then only keep alphabets in the string
clean_reviews = []
for review in reviews:
    s = nltk.word_tokenize(review)
    s = [word.lower() for word in s]
    s = [word for word in s if not word in set(
        nltk.corpus.stopwords.words('english'))]
    s = [stemmer.stem(word) for word in s if word.isalpha()]
    clean_reviews.append(s)

In [6]:
len(clean_reviews)

147

In [7]:
clean_reviews

[['generat',
  'san',
  'franciscan',
  'even',
  'though',
  'member',
  'famili',
  'also',
  'own',
  'restaur',
  'pier',
  'favorit'],
 ['jeff',
  'owner',
  'j',
  'carrillo',
  'plumb',
  'run',
  'honest',
  'fair',
  'famili',
  'own',
  'busi',
  'trust',
  'get',
  'job',
  'done',
  'right',
  'high',
  'recommend',
  'servic'],
 ['thank',
  'plumb',
  'hero',
  'victor',
  'come',
  'provid',
  'servic',
  'fix',
  'home',
  'main',
  'water',
  'pipe',
  'plumb',
  'hero',
  'assess',
  'issu',
  'time',
  'manner',
  'clear',
  'work',
  'need',
  'done',
  'short',
  'cut',
  'taken',
  'plumb',
  'hero',
  'also',
  'generous',
  'quot',
  'provid',
  'anyon',
  'experi',
  'main',
  'water',
  'pipe',
  'issu',
  'etc',
  'reach',
  'plumb',
  'hero',
  'avail'],
 ['high',
  'recommend',
  'let',
  'dog',
  'lauren',
  'team',
  'absolut',
  'wonder',
  'profession',
  'client',
  'dog',
  'love',
  'dog',
  'pack',
  'much',
  'fun',
  'dog',
  'park',
  'also',
  't

In [8]:
unique_words = set([word for review in clean_reviews for word in review])

In [10]:
len(unique_words)

1444

In [17]:
nwz = np.random.rand(5, 3)

In [19]:
pwz = nwz/nwz.sum(axis=0,keepdims=1)

In [20]:
pwz

array([[0.27848297, 0.00237072, 0.09467207],
       [0.0398961 , 0.11492877, 0.01949206],
       [0.26152606, 0.15779091, 0.16567802],
       [0.32691769, 0.4539953 , 0.44694814],
       [0.09317718, 0.2709143 , 0.27320972]])

In [21]:
nzd = np.random.rand(3, 2)
pzd = nzd/nzd.sum(axis=0,keepdims=1)

In [23]:
pzd

array([[0.41840696, 0.32679751],
       [0.38544574, 0.28555209],
       [0.1961473 , 0.38765041]])

In [34]:
0.00237072*0.38544574

0.0009137839247328

In [24]:
pzdw = np.zeros((3, 2, 5))

In [25]:
pzdw

array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]])

In [26]:
pwd = np.matmul(pwz, pzd)

In [29]:
pwd

array([[0.13600267, 0.12838417],
       [0.06481492, 0.0534122 ],
       [0.20274145, 0.19474874],
       [0.39944286, 0.40973482],
       [0.19699809, 0.21372007]])

In [32]:
np.outer(pwz, pzd)

array([[0.11651921, 0.09100754, 0.10734008, 0.07952139, 0.05462368,
        0.10795404],
       [0.00099193, 0.00077475, 0.00091378, 0.00067696, 0.00046501,
        0.00091901],
       [0.03961145, 0.0309386 , 0.03649094, 0.02703381, 0.01856967,
        0.03669967],
       [0.01669281, 0.01303795, 0.01537778, 0.01139241, 0.00782551,
        0.01546574],
       [0.048087  , 0.03755843, 0.0442988 , 0.03281815, 0.02254297,
        0.04455218],
       [0.00815561, 0.00636996, 0.00751313, 0.005566  , 0.00382332,
        0.00755611],
       [0.10942432, 0.08546606, 0.1008041 , 0.07467931, 0.05129763,
        0.10138068],
       [0.06602082, 0.05156568, 0.06081984, 0.04505752, 0.03095026,
        0.06116771],
       [0.06932083, 0.05414316, 0.06385988, 0.0473097 , 0.0324973 ,
        0.06422515],
       [0.13678464, 0.10683589, 0.12600903, 0.09335203, 0.06412402,
        0.12672978],
       [0.18995479, 0.14836453, 0.17499055, 0.1296393 , 0.08904995,
        0.17599146],
       [0.18700621, 0

In [35]:
a = np.array([1,2,3])
b = np.array([4,5,6])

In [36]:
np.dot(a,b)

32

In [38]:
np.multiply(a,b) / 3

array([1.33333333, 3.33333333, 6.        ])